# RAG Based PDF Question Answering System

## Step 1: Install Dependencies
## Step 2: Import Libraries
## Step 3: Load PDF
## Step 4: Split into Chunks
## Step 5: Create Embeddings
## Step 6: Store in ChromaDB
## Step 7: Create Retriever
## Step 8: Load LLM
## Step 9: Ask Questions






dependencies

In [ ]:
!pip install -U langchain langchain-community chromadb pypdf sentence-transformers transformers accelerate

import libraries

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.llms import HuggingFacePipeline

from transformers import pipeline

file loading

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
loader = PyPDFLoader("AML FINAL NOTES.pdf")
docs = loader.load()

print("Total Pages:", len(docs))
print("\nSample Content:\n")
print(docs[0].page_content[:300])

spliting into chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = text_splitter.split_documents(docs)

print("Total Chunks:", len(chunks))

embeddings

In [ ]:
embedding_function = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

create vector db

In [ ]:
persist_directory = "db"

vdb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory=persist_directory
)

vdb.persist()

print("Vector DB Created Successfully")

create retriever

In [ ]:
retriever = vdb.as_retriever(search_kwargs={"k": 5})

loading llm

In [ ]:
pipe = pipeline(
    "text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256,
    max_length=None
)

llm = HuggingFacePipeline(pipeline=pipe)

rag query

In [ ]:
query = "What are the final clusters in the K-means example?"

retrieved_docs = retriever.invoke(query)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
Answer the question based only on the context below:

Context:
{context}

Question:
{query}
"""

response = llm.invoke(prompt)

print(response)

In [ ]:
#some more questions


query = "which algorithm in this document use distance calculations?"

retrieved_docs = retriever.invoke(query)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
Answer the question based only on the context below:

Context:
{context}

Question:
{query}
"""

response = llm.invoke(prompt)

print(response)

In [ ]:
#some more questions


query = "which Clustering methods are explained in the document"

retrieved_docs = retriever.invoke(query)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
Answer the question based only on the context below:

Context:
{context}

Question:
{query}
"""

response = llm.invoke(prompt)

print(response)